In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'f:\\Generative\\Practice\\env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [5]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

In [6]:
api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
ar_tool = ArxivQueryRun(api_wrapper=api_wrapper)

In [8]:
ar_tool

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
loader = WebBaseLoader("https://www.britannica.com/technology/artificial-intelligence")
docs=loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                          chunk_overlap=200)
splitted_docs = splitter.split_documents(docs)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(splitted_docs,embeddings)
retriever = db.as_retriever()

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "AI_Search",
    "Search for information about Artificial Intelligence. For any questions about AI or ML, you must use this tool!",
)

In [14]:
tools = [wiki_tool, ar_tool, retriever_tool]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'f:\\Generative\\Practice\\env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='AI_Search', description='Search for information about Artificial Intelligence. For any questions about AI or ML, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026DA37D0B80>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langch

In [22]:
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

In [23]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

f:\Generative\Practice\env\Lib\site-packages\langsmith\client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
f:\Generative\Practice\env\Lib\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(model, tools, prompt)

In [25]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
agent_executor.invoke({"input":"What are Transformers?"})



> Entering new AgentExecutor chain...

Invoking: `AI_Search` with `{'query': 'Transformers AI'}`


(Show more)





						See all related content →




	Ask the Chatbot a Question



	Ask the Chatbot a Question


				Top Questions
			 What is artificial intelligence? Artificial intelligence is the ability of a computer or computer-controlled robot to perform tasks that are commonly associated with the intellectual processes characteristic of humans, such as the ability to reason. Although there are as yet no AIs that match full human flexibility over wider domains or in tasks requiring much everyday knowledge, some AIs perform specific tasks as well as humans. Learn more.  Are artificial intelligence and machine learning the same? No, artificial intelligence and machine learning are not the same, but they are closely related. Machine learning is the method to train a computer to learn from its inputs but without explicit programming for every circumstance. Machine learning helps a co

{'input': 'What are Transformers?',
 'output': 'The Transformers architecture is a type of deep learning model that uses self-attention mechanisms to process input sequences. It was first introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. The model consists of an encoder and a decoder, and it uses a self-attention mechanism to weigh the importance of different input elements when generating the output sequence.\n\nThe Transformer architecture has several key components:\n\n1. Self-Attention Mechanism: This is the core component of the Transformer architecture. It allows the model to weigh the importance of different input elements when generating the output sequence.\n2. Encoder: The encoder is responsible for processing the input sequence and generating a set of vectors that represent the input.\n3. Decoder: The decoder is responsible for generating the output sequence based on the vectors generated by the encoder.\n4. Positional Encoding: This is a mechani

In [29]:
answer=agent_executor.invoke({"input":"What is QLORA technique of quantization?"})
print(f"Question:\n{answer['input']}")
print(f"Answer:\n{answer['output']}")



> Entering new AgentExecutor chain...
<arxiv>{"query": "QLORA technique of quantization"}</arxiv>

> Finished chain.
Question:
What is QLORA technique of quantization?
Answer:
<arxiv>{"query": "QLORA technique of quantization"}</arxiv>
